In [ ]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

def append_to_path(dir0): # A convenience function
    if dir0 not in sys.path:
        sys.path.append(dir0)

# standard imports
import os, sys, json
import numpy as np
from matplotlib import pyplot as plt

# mountainlab imports
from mountainlab_pytools import mlproc as mlp
from mountainlab_pytools import mdaio
import spikeforestwidgets as SFW

# imports from this repo
append_to_path(os.getcwd()+'/../../python')
from synthesize_dataset import synthesize_dataset # Synthesize a test dataset
from validate_sorting_results import validate_sorting_results # Validation processors

In [ ]:
#######################################
# Initialize the pipeline object
#######################################

Pipeline=mlp.initPipeline()

In [ ]:
# Make synthetic ephys data and create output directory
dsdir=os.getcwd()+'/dataset'
with Pipeline:
    synthesize_dataset(dsdir,M=4,duration=60,average_snr=8,K=5)
    
output_base_dir=os.getcwd()+'/output'
if not os.path.exists(output_base_dir):
    os.mkdir(output_base_dir)

In [ ]:
Pipeline=mlp.initPipeline()

In [ ]:
#dsdir='dataset'
dsdir='kbucket://b5ecdf1474c5/datasets/neuron_paper/synth_K30'

with Pipeline:
    mlp.addProcess(
        'spyking_circus.sort',
        dict(
            timeseries=dsdir+'/raw.mda',
            geom=dsdir+'/geom.csv'
        ),
        dict(
            firings_out='output/firings.mda'
        ),
        dict(
            samplerate=30000,
            spike_thresh=4,
            detect_sign=1,
            adjacency_radius=30
        )
    )

In [ ]:
Pipeline=mlp.initPipeline()

In [ ]:
with Pipeline:
    A=validate_sorting_results(dataset_dir=dsdir,sorting_output_dir='output',output_dir='output')
    amplitudes_true=A['amplitudes_true']
    accuracies=A['accuracies']

In [ ]:
# Plot the comparison with ground truth
plt.plot(amplitudes_true,accuracies,'.')
plt.xlabel('Amplitude')
plt.ylabel('Accuracy');
plt.title('Accuracy vs. amplitude for {}'.format('simulated data'))